In [18]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from subprocess import check_output
import csv
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.svm import SVC
from sklearn import cross_validation
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from math import sqrt
%matplotlib inline
from sklearn.model_selection import KFold
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor

In [21]:
group_data = pd.read_csv('../output/result_regression.csv')
specific_data = pd.read_csv('../output/final_data.csv')

# Merge two datasets into one by key 'Topic_Year'
final_data = group_data.merge(specific_data, how='right', on='Topic_Year')

del group_data
del specific_data

# Filter data after year 2000
final_data = final_data[(final_data.PY >= 2004) & (final_data.PY < 2019)]
final_data.drop(columns=['PY','Cluster_Topic'], inplace=True)

In [22]:
# Missing data imputation by replacing median values
missing_cols = []
for col in final_data.columns:
    if final_data[col].isnull().sum() != 0:
        missing_cols.append(col)
for missing_col in missing_cols:
    final_data[missing_col].fillna(np.nanmedian(final_data[missing_col]), inplace=True)

year_2018 = final_data['Year'] == 2018
topic = final_data['Topic']
final_data.drop(columns=['Year','Topic','Topic_Year','Growth_Rate'], inplace=True)

In [24]:
le = preprocessing.LabelEncoder()
final_data['Squeezed_Publisher'] = le.fit_transform(final_data['Squeezed_Publisher'])

In [25]:
final_data['PT'] = le.fit_transform(final_data['PT'])

In [12]:
y = final_data['Target'].values
df = final_data.drop(['Target'],axis=1)
X = df.values

kf = KFold(n_splits=5)
rst = []
for train_index, test_index in kf.split(X):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clf = Ridge(alpha=1.0)
    clf.fit(X_train, y_train) 
    rst.append(sqrt(mean_squared_error(clf.predict(X_test),y_test)))
print('5 fold RMSE for ridge regression is :',sum(rst) / len(rst))

5 fold RMSE for ridge regression is : 0.1107929351166554


In [13]:
y = final_data['Target'].values
df = final_data.drop(['Target'],axis=1)
X = df.values

features = final_data.columns.tolist()
features.remove('Target')

kf = KFold(n_splits=5)
rst = []
categorical = ['Squeezed_Publisher','PT']

lgbm_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    # 'max_depth': 15,
    'num_leaves': 270,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.75,
    'bagging_freq': 2,
    'learning_rate': 0.01,
    'verbose': 1
}

for train_index, test_index in kf.split(X):
    lgtrain = lgb.Dataset(X_train, y_train,
                          feature_name=features,
                          categorical_feature=categorical)
    lgvalid = lgb.Dataset(X_test, y_test,
                          feature_name=features,
                          categorical_feature=categorical)


    # Go Go Go
    lgb_clf = lgb.train(
        lgbm_params,
        lgtrain,
        num_boost_round=20000,
        valid_sets=[lgtrain, lgvalid],
        valid_names=['train', 'valid'],
        early_stopping_rounds=1000,
        verbose_eval=100
    )
    rst.append(np.sqrt(metrics.mean_squared_error(y_test, lgb_clf.predict(X_test))))
print('5 fold RMSE for lightgbm is :',sum(rst) / len(rst))

C:\Users\lin\Anaconda3\lib\site-packages\lightgbm\basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\Users\lin\Anaconda3\lib\site-packages\lightgbm\basic.py:681: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


Training until validation scores don't improve for 1000 rounds.
[100]	train's rmse: 0.0498872	valid's rmse: 0.131301
[200]	train's rmse: 0.0182714	valid's rmse: 0.12957
[300]	train's rmse: 0.0066924	valid's rmse: 0.129838
[400]	train's rmse: 0.0024527	valid's rmse: 0.13006
[500]	train's rmse: 0.000899681	valid's rmse: 0.130135
[600]	train's rmse: 0.000332823	valid's rmse: 0.130186
[700]	train's rmse: 0.000128972	valid's rmse: 0.130204
[800]	train's rmse: 6.07133e-05	valid's rmse: 0.130209
[900]	train's rmse: 4.16208e-05	valid's rmse: 0.13021
[1000]	train's rmse: 3.58815e-05	valid's rmse: 0.13021
[1100]	train's rmse: 3.30506e-05	valid's rmse: 0.130209
Early stopping, best iteration is:
[198]	train's rmse: 0.0186424	valid's rmse: 0.129503
Training until validation scores don't improve for 1000 rounds.
[100]	train's rmse: 0.0498872	valid's rmse: 0.131301
[200]	train's rmse: 0.0182714	valid's rmse: 0.12957
[300]	train's rmse: 0.0066924	valid's rmse: 0.129838
[400]	train's rmse: 0.0024527	v

In [16]:
rst = []
for train_index, test_index in kf.split(X):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clf = RandomForestRegressor()
    clf.fit(X_train, y_train) 
    rst.append(sqrt(mean_squared_error(clf.predict(X_test),y_test)))
print('5 fold RMSE for RandomForest is :',sum(rst) / len(rst))

5 fold RMSE for RandomForest is : 0.1387998350700414


In [19]:
rst = []
for train_index, test_index in kf.split(X):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clf = AdaBoostRegressor()
    clf.fit(X_train, y_train) 
    rst.append(sqrt(mean_squared_error(clf.predict(X_test),y_test)))
print('5 fold RMSE for AdaBoost is :',sum(rst) / len(rst))

5 fold RMSE for AdaBoost is : 0.11887119656585834


In [38]:
y = final_data['Target'].values
df = final_data.drop(['Target'],axis=1)
X = df.values

X_train, X_test = X[~year_2018], X[year_2018]
y_train, y_test = y[~year_2018], y[year_2018]

clf = Ridge(alpha=1.0)
clf.fit(X_train, y_train) 
rst.append(sqrt(mean_squared_error(clf.predict(X_test),y_test)))

print('5 fold RMSE for ridge regression is :',sum(rst) / len(rst))

5 fold RMSE for ridge regression is : 0.1279722882164092


In [44]:
prediction_2018 = pd.DataFrame({'Predict':clf.predict(X_test),'Topic':topic[year_2018]})

In [51]:
prediction_2018.groupby(['Topic'])['Predict'].mean().reset_index().sort_values(by=['Predict'],ascending=False)

,Topic,Predict
9,9.0,0.310611
7,7.0,0.099569
2,2.0,0.097580
3,3.0,0.067479
4,4.0,0.060595
5,5.0,0.031080
14,14.0,0.001260
6,6.0,-0.015415
10,10.0,-0.047980
13,13.0,-0.054274
